# Damn You Bobby!

You're so good.

How do you win so much?! 

What a dick.

I love you.

In [1]:
from IPython.display import Image
# from IPython.core.display import HTML 
Image(url= "https://www.fb101.com/wp-content/uploads/2015/05/Boby-Flay.jpeg")

In [2]:
# Business as usual - standard imports and settings - nothing too exciting here

# do this - else integer division
from __future__ import division

# the web tools
from urllib import urlopen
from bs4 import BeautifulSoup

# default dictionaries are awesome!
from collections import defaultdict

# natural language processing
import nltk
import re

# library for getting p(gender) from first name
import genderize

# essentials
import numpy as np
import pandas as pd

# plotting
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

# machine learning
import sklearn


# Settings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.set_option('display.max_columns', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

# set the win/loss color scheme
colors = {0: 'red', 1: 'blue'}

AttributeError: 'module' object has no attribute 'style'

The sweet sweet data we are interested in is on Wikipedia (surprise)... 

https://en.wikipedia.org/wiki/Bobby_Flay

In particular, there are tables of data for each season of the show, that include the names of the contestants, the names of the guest hosts, the names of the judges, the opening round ingredient, the challenge dish, the winner, and the air date for each episode. There are over 100 episodes cataloged on Wikipeida. Importantly, someone(s) is actively updating the data. Given the dynamic nature of the data, I opted to write a web scraper to collect the data as needed rather than collect the data once manually (e.g. copy and paste).

In [ ]:
Image(filename=r'/home/troyer/Documents/0_projects/beat-bobby-flay/images/Bobby_Wiki.jpg')

In [ ]:
# request the webpage content
try:
    html = urlopen(r'https://en.wikipedia.org/wiki/Beat_Bobby_Flay')
except:
    print 'error opening url'

# parse the content into a BeautifulSoup object
try:
    soup = BeautifulSoup(html, 'html.parser')
except:
    print 'error reading html'

At this point we have the entire webpage content. What we want is just the data contained within each season table.
Within the HTML the tables are identified with the <table> tag, and the rows within the tables are identified with the <tr> tag. The catch is that there are also other tables (and rows) on the page that we don't care about. Importantly, the table rows we are interested in all start with two integer fields (show ID, season ID); everything else is junk. Thus, all we need to do is read all the tables and check if the first two values in each row is are show ID and season ID- if they are, keep the row - else move on. 

In [ ]:
# is first row value int-able? The values are stored as text,
# so we want to see if we can render it as an integer 
# - does it quack like an integer?

def is_int(s):
    try:
        int(s)
        return True
    except ValueError:
        return False
    
# Determines if a table_row is a cookoff match entry
def is_cookoff(table_row):
    row_cells = table_row.findAll("td")
    if len(row_cells) < 2:
        return False
    else:
        # first two non-empty cells must be int - able
        return (is_int(row_cells[0].text) and 
                is_int(row_cells[1].text))

def get_matches(soup):
    matches = []
    # locate all the rows
    all_rows_in_page = soup.findAll("tr")
    for row in all_rows_in_page:
        # check if first two values are IDs
        if is_cookoff(row):
            # if we find a good row, pick out all the data fields - <td>
            row_cells = row.findAll("td")
            cookoff_matches = {
                "Show_ID":           row_cells[0].text,
                "Season_ID":         row_cells[1].text,
                "Title":             row_cells[2].text,
                "Original Airdate":  row_cells[3].text,
                "Guest(s)":          row_cells[4].text,
                "Ingredient(s)":     row_cells[5].text,
                "Contestants":       row_cells[6].text,
                "Judges":            row_cells[7].text,
                "Dish":              row_cells[8].text,
                # Challenger is not its own field, the winner of the first
                # round is designated in bold.. :/
                "Challenger":        row_cells[6].findAll('b')[0].text \
                                         if row_cells[6].findAll('b')
                                         else 'Null',
                "Winner":            row_cells[9].text
            }
            matches.append(cookoff_matches)
    return matches

In [ ]:
# get it
cookoff_data = get_matches(soup)

# handoff to pandas because that's just what we do
df = pd.DataFrame(cookoff_data)
# make the overall id the index
df.index = df.Show_ID

# there are some empty place holder fields in the wiki table
# as is is actively being editted - ditch em
df = df[df.Contestants <> '']

# contestants is a compound text field seperated with a comma
df['Contestant_1'], df['Contestant_2'] = \
    df.Contestants.str.split(',').str

df['Guest(s)'] = df['Guest(s)'].str.replace(';', ',')
df['Guest_1'] = df['Guest(s)'].str.split(',').str[0]

df['Guest_2'] = np.where(df['Guest(s)'].str.split(',').str.len() > 1, 
                         df['Guest(s)'].str.split(',').str[1], 'NaN')

df['Winner'] = df['Winner'].str.replace('Booby', 'Bobby')

df['Judges'] = df['Judges'].str.replace('Greg and', 'Greg Bresnitz,')
df['Judges'] = df['Judges'].str.replace(', , ', ', ')

df['Judge_1'], df['Judge_2'], df['Judge_3'] = \
    df.Judges.str.split(',').str

df['Original_Airdate'] = \
    df['Original Airdate'].str.extract(r"\((.*?)\)", expand=False)

df['Bobby_Win'] = np.where(df.Winner == 'Bobby Flay', 1, 0)

df.drop(['Show_ID', 'Season_ID', 'Title', 'Contestants', 
         'Contestant_1', 'Contestant_2', 'Judges', 
         'Original Airdate', 'Original_Airdate',
         'Guest(s)', 'Ingredient(s)', 'Winner'],
          axis=1, inplace=True)

In [ ]:
fields_to_genderize =  ['Guest_1', 'Guest_2',
                        'Judge_1', 'Judge_2', 'Judge_3',
                        'Challenger']

gndr = gender.Detector(case_sensitive=False)

for f in fields_to_genderize:
    df['{}_gender'.format(f)] = df[f].str.strip().str.split(' ').str[0]
    df['{}_gender'.format(f)] = \
        df['{}_gender'.format(f)].apply(lambda x: gndr.get_gender(x))
        
df.replace(['mostly_female', 'mostly_male'],
           ['female', 'male'], inplace=True)

df.replace(['female', 'male'],[0, 1], inplace=True)

def random_andy(val):
    if val == 'andy':
        return np.random.randint(0,2)
    else: return val
    
for col in df.columns:
    df[col] = df[col].apply(random_andy)

df['Guest_mean_gender'] = \
    (df.Guest_1_gender + df.Guest_2_gender) / 2
    
df['Judge_mean_gender'] = \
    (df.Judge_1_gender + df.Judge_2_gender + df.Judge_3_gender) / 3
    
df.drop(['{}_gender'.format(f) for f in fields_to_genderize 
         if f not in ('Challenger')], axis=1, inplace=True)

df.drop(['Challenger', 
         'Guest_1', 'Guest_2', 
         'Judge_1', 'Judge_2', 'Judge_3'],
         axis=1, inplace=True)

In [ ]:
df.groupby('Bobby_Win').mean().T

In [ ]:
cm = df[['Guest_mean_gender', 'Judge_mean_gender',
         'Challenger_gender', 'Bobby_Win']].corr()

_, ax = plt.subplots(figsize=(13,10)) 

# graph correlation matrix
_ = sns.heatmap(cm, ax=ax,
    xticklabels=cm.columns.values,
    yticklabels=cm.columns.values)

In [ ]:
cm = df.corr()

_, ax = plt.subplots(figsize=(13,10)) 

# graph correlation matrix
_ = sns.heatmap(cm, ax=ax,
    xticklabels=cm.columns.values,
    yticklabels=cm.columns.values)

# Word vector dish type

In [ ]:
dish_strings = df.Dish.dropna()
terms = []

for item in dish_strings:
    splits = re.sub('[^a-zA-Z-]', ' ', item.encode('utf-8')).split(' ')
    terms.extend(splits)
    
terms = list(set(term for term in terms if len(term) > 3))

def word_freq_processor(compare_words, comparison_texts):

    stemmer = nltk.stem.porter.PorterStemmer()  

    def stem_tokens(tokens, stemmer):
        sts = [stemmer.stem(item) for item in tokens]
        return sts

    def tokenize(text):
        return stem_tokens(nltk.word_tokenize(text), stemmer)

    vectorizer = sklearn.feature_extraction.text.CountVectorizer(
                     tokenizer=tokenize, stop_words='english')

    vectored = vectorizer.fit(compare_words)
    targets = [vectored.transform([item]) for item in comparison_texts]
    targets = [item.toarray()[0] for item in targets]
    cols = vectored.get_feature_names()
    word_df = pd.DataFrame.from_records(targets, columns=cols)
    return word_df

dish_df = word_freq_processor(terms, dish_strings)

df = df.reset_index().join(dish_df)

df.drop(['Show_ID', 'Dish'], axis=1, inplace=True)

# Classification

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.grid_search     import GridSearchCV

from sklearn.neighbors       import KNeighborsClassifier
from sklearn.ensemble        import RandomForestClassifier
from sklearn.ensemble        import GradientBoostingClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.neural_network  import MLPClassifier
from sklearn.svm             import LinearSVC
from sklearn.svm             import SVC
from sklearn.naive_bayes     import GaussianNB
from sklearn.naive_bayes     import BernoulliNB
from sklearn.naive_bayes     import MultinomialNB

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Remove the target column from df
targets = df.Bobby_Win
df.drop(['Bobby_Win'], axis=1, inplace=True)

# set aside for later
X_train, X_test, y_train, y_test = \
    train_test_split(df, targets, train_size = 0.7)

In [ ]:
KNrNgb = KNeighborsClassifier()
RandFr = RandomForestClassifier()
GrdtBC = GradientBoostingClassifier()
LogReg = LogisticRegression()
nnwMLP = MLPClassifier()
LinSVC = LinearSVC()
svmSVC = SVC()

classifiers = {
             'K Nearest Neighbors'                   : KNrNgb,
             'Random Forest'                         : RandFr,
             'Gradient Boosted Decision Trees'       : GrdtBC,
             'Logistic Regression'                   : LogReg,
             'Neural Network Multi-layer Perceptron' : nnwMLP,
             'Linear Support Vector Classifier'      : LinSVC,
             'Support Vector Machine'                : svmSVC
              }

print ' Model Results '.center(60, '-')

performance = defaultdict(float)

for name, model in classifiers.items():

    scores = cross_val_score(model, X_train, y_train, cv=5)
    print '\n{}:\n\n\tAccuracy: {:.3f} (+/- {:.3f})'\
          ''.format(name, scores.mean(), scores.std() * 2)

    performance[name] = scores.mean()

    print
    print '\t\t', 'Parameters'.center(45, '-')
    for k, v in sorted(model.get_params().items()):
        if len(str(v)) > 15:
            v = str(v)[:15]
        print '\t\t|', k.ljust(25, '.'), str(v).rjust(15, '.'), '|'
    print '\t\t', ''.center(45, '-')
    print
    
    top_model   = max(performance.items(), key=lambda x: x[1])
    worst_model = min(performance.items(), key=lambda x: x[1])

In [ ]:
# The model params to grid search

low = np.linspace(0.00001, 1, 3)
high = np.linspace(1.00001, 10000, 3)
C_ = low.tolist()
C_.append(high.tolist())

parameters = {
       
'K Nearest Neighbors':                  
    {'algorithm'                : ['ball_tree', 'kd_tree', 'brute'],
     'leaf_size'                : [1],
     'metric'                   : ['manhattan', 'euclidean', 
                                   'chebyshev', 'seuclidean', 
                                   'mahalanobis'],
     'n_neighbors'              : [1, 3],
     'p'                        : [2],
     'weights'                  : ['distance']},  

'Random Forest':
    {'bootstrap'                : [True, False], 
     'criterion'                : ['entropy', 'gini'],  
     'max_depth'                : [None],
     'max_features'             : [None],   
     'max_leaf_nodes'           : [None],
     'min_impurity_split'       : [1e-07],
     'min_samples_leaf'         : [1],
     'min_samples_split'        : [2], 
     'min_weight_fraction_leaf' : [0.0],
     'n_estimators'             : [100], 
     'oob_score'                : [True, False]}, 

'Gradient Boosted Decision Trees':
    {'criterion'                : ['friedman_mse', 'mse', 'mae'], 
     'loss'                     : ['deviance', 'exponential'],    
     'learning_rate'            : [0.1, 1],
     'max_depth'                : [None, 10],
     'max_features'             : [None],
     'max_leaf_nodes'           : [None],
     'min_impurity_split'       : [1e-07],
     'min_samples_leaf'         : [1],
     'min_samples_split'        : [2], 
     'min_weight_fraction_leaf' : [0.0],
     'n_estimators'             : [200],
     'presort'                  : ['auto'],
     'subsample'                : [1]},

'Logistic Regression':
    {'C'                        : C_,
     'class_weight'             : [None],
     'dual'                     : [True],
     'fit_intercept'            : [True],
     'intercept_scaling'        : C_,
     'max_iter'                 : [400],
     'penalty'                  : ['l1', 'l2'],
     'solver'                   : ['newton-cg', 'lbfgs', 
                                   'liblinear', 'sag'],
     'tol'                      : [0.0001]},

'Neural Network Multi-layer Perceptron':
    {'activation'               : ['identity', 'relu', 
                                   'logistic', 'tanh'],
     'alpha'                    : C_, 
     'batch_size'               : ['auto'],
     'beta_1'                   : [0.9],   
     'beta_2'                   : [0.999], 
     'early_stopping'           : [False], 
     'epsilon'                  : [1e-08],
     'hidden_layer_sizes'       : [(100,)], 
     'learning_rate'            : ['constant', 'invscaling',
                                   'adaptive'],
     'learning_rate_init'       : C_, 
     'max_iter'                 : [200], 
     'momentum'                 : [0.9],        
     'nesterovs_momentum'       : [True],       
     'power_t'                  : [0.5],         
     'shuffle'                  : [True],
     'solver'                   : ['adam', 'lbfgs', 'sgd'],
     'tol'                      : [0.0001],
     'validation_fraction'      : [0.1]},

'Linear Support Vector Classifier':
   {'C'                         : C_,
    'class_weight'              : [None],  
    'dual'                      : [True],  
    'fit_intercept'             : [True],  
    'intercept_scaling'         : np.linspace(0.001, 1, 3),
    'loss'                      : ['hinge', 'squared_hinge'],
    'max_iter'                  : [200],
    'penalty'                   : ['l1', 'l2'],  
    'tol'                       : [0.0001]},

'Support Vector Machine':
    {'C'                        : C_,
     'cache_size'               : [200],
     'coef0'                    : np.linspace(0.001, 1, 3),
     'class_weight'             : [None],
     'degree'                   : range(0, 3), 
     'gamma'                    : ['auto'],
     'kernel'                   : ['linear', 'poly', 'rbf', 'sigmoid'],
     'max_iter'                 : [200],
     'probability'              : [True, False], 
     'shrinking'                : [True, False], 
     'tol'                      : [0.0001]}
    }

In [ ]:
# Pass along the top or bottom performing cold model for further tuning
final_classifier_name = top_model[0]  # Change
final_classifier = classifiers[final_classifier_name]

# Get a baseline for performance independent of the previous evaluation
scores = cross_val_score(final_classifier, X_train, y_train, cv=5)

print ' Model Cross-Validation '.center(60, '-')
print
print 'Top Model: {}'.format(final_classifier_name).center(60)
print
print
print ' {} Baseline '.format(final_classifier_name).center(60, '#')
print
print 'Accuracy: {:.3f} (+/- {:.3f})'.format(
           scores.mean(), scores.std() * 2)
print
for item in scores:
    print item

gscv = GridSearchCV(
    final_classifier, parameters[final_classifier_name],
    n_jobs=-1, cv=5, error_score=-1)

gscv.fit(X_train, y_train)

print
print "Best Score: {}".format(gscv.best_score_)
print
print "Best Params: {}".format(gscv.best_params_)
print;

In [ ]:
americas_next_top_model = eval(str(gscv.best_estimator_))
americas_next_top_model.fit(X_train, y_train)
predicted_y = americas_next_top_model.predict(X_test)
print
print " {} Final Trial ".format(final_classifier_name).center(60, '#')
print
print "Accuracy Score: {}".format(accuracy_score(y_test, predicted_y))
print
print "Confusion Matrix:"
print "|TN|FP|"
print "|FN|TP|"
print
print confusion_matrix(y_test, predicted_y)
print
print "Class Report:"
print classification_report(y_test, predicted_y);